In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis
import os
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import linregress
import warnings
#from tqdm import tqdm  # optional progress bar
warnings.filterwarnings('ignore')
import re
import math

## Description 

This notebook runs the code to analyze the MD simulation files. It calculates the diffusion coefficient of confined water molecules, the average number of hydrogen bonds per confined molecule, and the radial distribution of water molecules inside the tube. 

In [6]:
def load_trajectory(TOPO, TRAJ):
    """Load MD trajectory and select water oxygen atoms."""
    u = mda.Universe(TOPO, TRAJ)
    waters = u.select_atoms("resname SOL and name OW")
    return u, waters


def identify_initial_inside_waters(u, waters, cnt_center, R_tube, z_min, z_max):
    """Find which waters are inside the CNT at frame 0."""
    def inside_CNT(positions):
        x, y, z = positions[:, 0], positions[:, 1], positions[:, 2]
        r = np.sqrt((x - cnt_center[0])**2 + (y - cnt_center[1])**2)
        return np.where((r < R_tube) & (z > z_min) & (z < z_max))[0]

    u.trajectory[500]
    inside_indices = inside_CNT(waters.positions)
    inside_resids = np.unique(waters[inside_indices].resids)
    print(f"{len(inside_resids)} water molecules initially inside CNT")
    return inside_resids


def track_water_positions(u, inside_resids):
    """Track 3D positions of selected water molecules across all frames."""
    time = []
    positions = []
    for ts in u.trajectory:
        time.append(ts.time)
        res_pos = []
        for resid in inside_resids:
            res = u.select_atoms(f"resid {resid} and name OW")
            if len(res) == 0:
                continue
            res_pos.append(res.positions[0])
        positions.append(np.array(res_pos))

    positions = np.array(positions)  # (n_frames, n_mols, 3)
    time = np.array(time)
    positions /= 10.0  # Convert Å → nm
    return time, positions



def compute_radial_z_positions(positions, cnt_center, R_tube, z_min, z_max):
    """Compute instantaneous radial and z positions for each molecule."""
    cnt_center_nm = cnt_center / 10.0
    R_tube_nm = R_tube / 10.0
    z_min_nm, z_max_nm = z_min / 10.0, z_max / 10.0

    n_frames, n_mols, _ = positions.shape
    radii = np.zeros((n_frames, n_mols))
    z_positions = np.zeros((n_frames, n_mols))

    for f in range(n_frames):
        x, y, z = positions[f, :, 0], positions[f, :, 1], positions[f, :, 2]
        radii[f, :] = np.sqrt((x - cnt_center_nm[0])**2 + (y - cnt_center_nm[1])**2)
        z_positions[f, :] = z

    return radii, z_positions


def compute_msd_confined(positions, radii, z_positions, cnt_center, R_tube, z_min, z_max):
    """Compute MSD for confined water molecules that never escape CNT."""
    n_frames, n_mols, _ = positions.shape
    msd_dynamic_3D = np.zeros(n_frames)
    msd_dynamic_z  = np.zeros(n_frames)

    cnt_center_nm = cnt_center / 10.0
    R_tube_nm = R_tube / 10.0
    z_min_nm, z_max_nm = z_min / 10.0, z_max / 10.0
    active_mask = np.ones(n_mols, dtype=bool)

    used_positions_start = []
    used_positions_end   = []

    print("\nComputing MSD (confined water only, permanently removing escaped molecules)...")

    for dt in range(n_frames):
        displacements_3D = []
        displacements_z  = []

        for t0 in range(n_frames - dt):
            inside_both = (
                active_mask &
                (radii[t0] < R_tube_nm) & (radii[t0 + dt] < R_tube_nm) &
                (z_positions[t0] > z_min_nm) & (z_positions[t0] < z_max_nm) &
                (z_positions[t0 + dt] > z_min_nm) & (z_positions[t0 + dt] < z_max_nm)
            )

            if np.any(inside_both):
                # Save positions for visualization
                used_positions_start.append(positions[t0, inside_both])
                used_positions_end.append(positions[t0 + dt, inside_both])

                delta = positions[t0 + dt, inside_both] - positions[t0, inside_both]
                disp2_3D = np.sum(delta**2, axis=1)
                disp2_z  = delta[:, 2]**2
                displacements_3D.append(np.mean(disp2_3D))
                displacements_z.append(np.mean(disp2_z))

            # Deactivate escaped molecules
            escaped_now = (radii[t0 + dt] >= R_tube_nm)
            active_mask[escaped_now] = False

        if displacements_3D:
            msd_dynamic_3D[dt] = np.mean(displacements_3D)
            msd_dynamic_z[dt]  = np.mean(displacements_z)

    if used_positions_start:
        used_positions_start = np.vstack(used_positions_start)
        used_positions_end   = np.vstack(used_positions_end)
    else:
        used_positions_start = np.empty((0, 3))
        used_positions_end   = np.empty((0, 3))

    return msd_dynamic_3D, msd_dynamic_z, used_positions_start, used_positions_end


def fit_diffusion_coefficients(time, msd_dynamic_3D, msd_dynamic_z, fit_start, fit_end):
    """Fit MSD vs time to extract D3D and Dz in nm²/ps and cm²/s."""
    n_points = min(len(time), len(msd_dynamic_3D))
    time = time[:n_points]
    msd_dynamic_3D = msd_dynamic_3D[:n_points]
    msd_dynamic_z  = msd_dynamic_z[:n_points]

    mask = (time >= fit_start) & (time <= fit_end)

    slope_3D, intercept_3D, _, _, _ = linregress(time[mask], msd_dynamic_3D[mask])
    D_3D = slope_3D / 6.0
    D_3D_cm2_s = D_3D * 1e-2

    slope_z, intercept_z, _, _, _ = linregress(time[mask], msd_dynamic_z[mask])
    D_z = slope_z / 2.0
    D_z_cm2_s = D_z * 1e-2

    print("\n--- Confined-water MSD (permanent escape removal) ---")
    print(f"3D D = {D_3D:.3e} nm²/ps  ({D_3D_cm2_s:.3e} cm²/s)")
    print(f"z-axis D = {D_z:.3e} nm²/ps  ({D_z_cm2_s:.3e} cm²/s)")

    return D_3D, D_3D_cm2_s, D_z, D_z_cm2_s, slope_3D, intercept_3D, slope_z, intercept_z, mask


def save_diffusion_results(path, time, msd_dynamic_3D, msd_dynamic_z, D_3D_cm2_s, D_z_cm2_s):
    """Save MSD data and diffusion coefficients to CSV."""
    os.makedirs(os.path.join(path, "Figures"), exist_ok=True)

    np.savetxt(
        os.path.join(path, "Figures", "msd_inside_finiteCNT.csv"),
        np.c_[time, msd_dynamic_3D, msd_dynamic_z],
        header="tau_ps, MSD_3D_nm2, MSD_z_nm2",
        delimiter=",", fmt="%.6f"
    )

    np.savetxt(
        os.path.join(path, "Figures", "diffusion_coef_z.csv"),
        np.c_[D_z_cm2_s],
        header="Diffusion_z (cm2/s)",
        delimiter=",", fmt="%.15f"
    )

    np.savetxt(
        os.path.join(path, "Figures", "diffusion_coef_3D.csv"),
        np.c_[D_3D_cm2_s],
        header="Diffusion_3D (cm2/s)",
        delimiter=",", fmt="%.15f"
    )


# ================================================
# 3️⃣ PLOT MSD FITS
# ================================================
def plot_msd(time, msd_dynamic_3D, msd_dynamic_z,
             slope_3D, intercept_3D, slope_z, intercept_z, mask, path):
    """Plot MSD and linear fits."""
    plt.figure(figsize=(6, 4))
    plt.plot(time, msd_dynamic_3D, label='MSD (3D)')
    plt.plot(time[mask], slope_3D * time[mask] + intercept_3D, 'r--', label='Fit (3D)')
    plt.plot(time, msd_dynamic_z, label='MSD (z-axis)')
    plt.plot(time[mask], slope_z * time[mask] + intercept_z, 'g--', label='Fit (z)')
    plt.xlabel('Time (ps)')
    plt.ylabel('MSD (nm²)')
    plt.legend(fontsize=12, loc='upper right')
    plt.title('Dynamic MSD: confined water inside CNT')
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "MSD_plot.png"), dpi=600, bbox_inches="tight")
    plt.close()


# ================================================
# 4️⃣ PLOT RADIAL AND AXIAL TRAJECTORIES
# ================================================
def plot_radial_axial_positions(time, radii, z_positions, R_tube_nm, z_min_core, z_max_core, path):
    """Plot radial and axial trajectories of water molecules."""
    n_mols = radii.shape[1]
    os.makedirs(os.path.join(path, "Figures"), exist_ok=True)

    idx_sample = np.random.choice(n_mols, size=min(50, n_mols), replace=False)

    # Radial distance plot
    plt.figure(figsize=(6,4))
    for i in idx_sample:
        plt.plot(time, radii[:, i], alpha=0.4)
    plt.axhline(R_tube_nm, color='r', linestyle='--', label='CNT radius')
    plt.xlabel('Time (ps)')
    plt.ylabel('Radial distance (nm)')
    plt.title('Radial position of initially confined waters')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "Radial_Distance.png"), dpi=600, bbox_inches="tight")
    plt.close()

    # Axial z-position plot
    plt.figure(figsize=(6,4))
    for i in idx_sample:
        plt.plot(time, z_positions[:, i], alpha=0.4)
    plt.axhline(z_min_core, color='r', linestyle='--')
    plt.axhline(z_max_core, color='r', linestyle='--')
    plt.xlabel('Time (ps)')
    plt.ylabel('z position (nm)')
    plt.title('Axial motion of confined waters')
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "Axial_Distance.png"), dpi=600, bbox_inches="tight")
    plt.close()


# ================================================
# 5️⃣ GENERATE 3D VISUALIZATION OF LAST FRAME
# ================================================
def plot_3D_snapshot(u, waters, cnt_center_nm, R_tube_nm, z_min_nm, z_max_nm,
                     z_min_core, z_max_core, path):
    """Generate 3D scatter visualization of water inside CNT."""
    print("\nGenerating 3D visualization for final frame...")
    u.trajectory[-1]
    pos_last = waters.positions / 10.0
    x, y, z = pos_last[:, 0], pos_last[:, 1], pos_last[:, 2]
    r = np.sqrt((x - cnt_center_nm[0])**2 + (y - cnt_center_nm[1])**2)
    inside_idx = np.where((r < R_tube_nm) & (z_min_core < z) & (z < z_max_core))[0]
    outside_idx = np.setdiff1d(np.arange(len(pos_last)), inside_idx)

    inside_pos = pos_last[inside_idx]
    outside_pos = pos_last[outside_idx]

    fig = plt.figure(figsize=(8,7))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(outside_pos[:,0], outside_pos[:,1], outside_pos[:,2], s=8, c='skyblue', alpha=0.3)
    ax.scatter(inside_pos[:,0], inside_pos[:,1], inside_pos[:,2], s=12, c='red', alpha=0.8)

    theta = np.linspace(0, 2*np.pi, 50)
    z_vals = np.linspace(z_min_nm, z_max_nm, 50)
    theta_grid, z_grid = np.meshgrid(theta, z_vals)
    x_grid = cnt_center_nm[0] + R_tube_nm * np.cos(theta_grid)
    y_grid = cnt_center_nm[1] + R_tube_nm * np.sin(theta_grid)
    ax.plot_surface(x_grid, y_grid, z_grid, color='gray', alpha=0.1, linewidth=0)

    ax.set_xlabel('x (nm)')
    ax.set_ylabel('y (nm)')
    ax.set_zlabel('z (nm)')
    ax.set_box_aspect([1,1,6])
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "Simulation_model.png"), dpi=600, bbox_inches="tight")
    plt.close()


# ================================================
# 6️⃣ COMPUTE RADIAL DENSITY DISTRIBUTION
# ================================================
def compute_radial_density(u, cnt_center_nm, R_tube_nm, z_min_nm, z_max_nm, path):
    """Compute and plot radial density profiles of water and carbon."""
    def radial_distances(universe, center_nm, selection):
        atoms = universe.select_atoms(selection)
        positions_nm = atoms.positions / 10.0
        r = np.sqrt((positions_nm[:,0] - center_nm[0])**2 +
                    (positions_nm[:,1] - center_nm[1])**2)
        return r

    u.trajectory[-1]
    r_oxy = radial_distances(u, cnt_center_nm, "resname SOL and name OW")
    r_carbon = radial_distances(u, cnt_center_nm, "not resname SOL")

    max_r_plot = R_tube_nm + 0.5
    bins = np.linspace(0, max_r_plot, 100)
    r_centers = 0.5 * (bins[1:] + bins[:-1])
    dr = bins[1] - bins[0]
    Lz = z_max_nm - z_min_nm

    shell_volumes = 2 * np.pi * r_centers * dr * Lz
    counts_oxy, _ = np.histogram(r_oxy, bins=bins)
    counts_carbon, _ = np.histogram(r_carbon, bins=bins)

    rho_oxy = counts_oxy / shell_volumes
    rho_carbon = counts_carbon / shell_volumes
    
    
    plt.figure(figsize=(7,5))
    plt.plot(r_centers, rho_oxy/np.max(rho_oxy), color='C0', lw=2, label='Water O')
    plt.plot(-r_centers, rho_oxy/np.max(rho_oxy), color='C0', lw=2)
    plt.plot(r_centers, rho_carbon/np.max(rho_carbon), color='C1', lw=2, label='CNT C')
    plt.plot(-r_centers, rho_carbon/np.max(rho_carbon), color='C1', lw=2)
    plt.xlabel('Radial distance (nm)')
    plt.ylabel('Normalized density')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "Radial_density_Last_Frame.png"), dpi=600, bbox_inches="tight")
    plt.close()

    rho_oxy_norm = rho_oxy / np.max(rho_oxy)
    rho_carbon_norm = rho_carbon / np.max(rho_carbon)

    # Create a single table (DataFrame)
    df = pd.DataFrame({
        "r_center": r_centers,
        "rho_oxy": rho_oxy,
        "rho_carbon": rho_carbon,
        "rho_oxy_norm": rho_oxy_norm,
        "rho_carbon_norm": rho_carbon_norm
    })

    # Save to CSV
    df.to_csv(os.path.join(path, "Figures", "density_profiles.csv"), index=False)

# ================================================
# 7️⃣ COMPUTE BULK WATER DENSITY OUTSIDE CNT
# ================================================
def compute_bulk_density(u, cnt_center_nm, R_tube_nm, path):
    """Compute average density of water molecules outside the CNT."""
    waters = u.select_atoms("resname SOL and name OW")
    M_water = 18.01528
    NA = 6.02214076e23
    nm3_to_cm3 = 1e-21

    densities = []
    times = []

    for ts in u.trajectory:
        Lx, Ly, Lz = ts.dimensions[:3]/10
        V_box = Lx * Ly * Lz

        pos = waters.positions / 10.0
        r = np.sqrt((pos[:,0] - cnt_center_nm[0])**2 + (pos[:,1] - cnt_center_nm[1])**2)
        outside_mask = r > R_tube_nm
        N_outside = np.sum(outside_mask)

        V_tube = np.pi * (R_tube_nm**2) * Lz
        V_outside = V_box - V_tube
        mass_g = N_outside * M_water / NA
        rho = mass_g / (V_outside * nm3_to_cm3)
        densities.append(rho)
        times.append(ts.time)

    rho_avg = np.mean(densities)
    print(f"Average bulk water density (outside CNT): {rho_avg:.3f} g/cm³")

    plt.plot(times, densities)
    plt.axhline(rho_avg, color='r', linestyle='--', label=f'Avg = {rho_avg:.3f} g/cm³')
    plt.xlabel("Time (ps)")
    plt.ylabel("Density (g/cm³)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(path, "Figures", "Weight_Density_of_Water.png"), dpi=600, bbox_inches="tight")
    plt.close()


def fit_MSD(path, time, msd3d, msdz, lower_time, upper_time):
    # ---- Choose linear region for fitting (e.g., 100–500 ps) ----
    mask = (time >= lower_time) & (time <= upper_time)
    time_fit = time[mask]
    msd_fit = msd3d[mask]

    # ---- Perform linear regression ----
    slope, intercept, r_value, p_value, std_err = linregress(time_fit, msd_fit)

    # ---- Compute diffusion coefficient ----
    # MSD = 6 * D * t  ->  D = slope / 6
    D_nm2_ps = slope / 6
    D_m2_s = D_nm2_ps * 1e-6  # convert to m^2/s
    D_cm2_s = D_m2_s *1e4

    print(f"Slope: {slope:.4f} nm²/ps")
    print(f"Diffusion coefficient: {D_nm2_ps:.4e} nm²/ps = {D_m2_s:.4e} m²/s = {D_cm2_s:.4e} cm²/s")

    np.savetxt(path + "Figures/" + "diffusion_coef_3D.csv", np.c_[D_cm2_s],
    header="Diffusion_3D Coef in cm2/s",
    delimiter=",", fmt="%.15f")

    ############################################ repeat for 3D ##############################
    mask = (time >= lower_time) & (time <= upper_time)
    time_fit = time[mask]
    msd_fit = msdz[mask]

    # ---- Perform linear regression ----
    slopez, interceptz, r_value, p_value, std_err = linregress(time_fit, msd_fit)

    # ---- Compute diffusion coefficient ----
    # MSD = 6 * D * t  ->  D = slope / 6
    D_nm2_ps = slope / 2
    D_m2_s = D_nm2_ps * 1e-6  # convert to m^2/s
    D_cm2_s = D_m2_s *1e4

    print(f"Slope: {slope:.4f} nm²/ps")
    print(f"Diffusion coefficient: {D_nm2_ps:.4e} nm²/ps = {D_m2_s:.4e} m²/s = {D_cm2_s:.4e} cm²/s")

    np.savetxt(path + "Figures/" + "diffusion_coef_z.csv", np.c_[D_cm2_s],
    header="Diffusion_z Coef in cm2/s",
    delimiter=",", fmt="%.15f")
    
    # ---- Plot ----
    fig, ax = plt.subplots(figsize=(7,7))
    plt.plot(time, msd3d, label='MSD 3D data')
    plt.plot(time, msdz, label='MSD Z data')
    #plt.plot(df2[:,0], df2[:,1], linewidth=3, label = '2000ps')
    plt.plot(time_fit, slope*time_fit + intercept, 'r--', label='Linear fit')
    plt.plot(time_fit, slopez*time_fit + interceptz, 'r--')
    plt.xlabel('Time (ps)')
    plt.ylabel('MSD (nm²)')
    plt.legend(fontsize=14)
    plt.title('MSD Fit')
    plt.savefig(path + "Figures" + "/MSD_plot.png", dpi=600, bbox_inches="tight")
    #plt.show()

def compute_confined_msd_random_with_traj(u, waters, cnt_center, R_tube, z_min, z_max,
                                          n_trials=100, n_mols_per_trial=20, seed=None,
                                          save_path="./Results"):
    """
    Compute MSD for confined water inside a CNT by randomly sampling time origins and molecules.
    Also saves and plots trajectories that remain inside the CNT for both t0 and t0+dt.

    Parameters
    ----------
    u : MDAnalysis.Universe
        Universe containing trajectory and topology.
    waters : AtomGroup
        Water oxygen atoms.
    cnt_center : np.ndarray
        CNT center (Å).
    R_tube : float
        CNT radius (Å).
    z_min, z_max : float
        CNT axial bounds (Å).
    n_trials : int
        Number of random time origins to sample.
    n_mols_per_trial : int
        Number of water molecules to sample per trial.
    seed : int or None
        Random seed for reproducibility.
    save_path : str
        Directory to save outputs (plots, trajectories).

    Returns
    -------
    time : np.ndarray
        Time array (ps).
    msd3D : np.ndarray
        Mean squared displacement (3D) for confined waters.
    msdZ : np.ndarray
        Mean squared displacement (z-axis) for confined waters.
    """

    rng = np.random.default_rng(seed)
    os.makedirs(save_path, exist_ok=True)

    cnt_center_nm = cnt_center / 10.0
    R_tube_nm = R_tube / 10.0
    z_min_nm, z_max_nm = z_min / 10.0, z_max / 10.0

    n_frames = len(u.trajectory)
    time = []
    for ts in u.trajectory:
        time.append(ts.time)
    time = np.array(time)

    # Convert positions for all waters at all frames
    all_positions = []
    for ts in tqdm(u.trajectory, desc="Reading trajectory"):
        all_positions.append(waters.positions / 10.0)  # Å → nm
    all_positions = np.array(all_positions)  # (n_frames, n_waters, 3)

    dx = all_positions[:,:,0] - cnt_center_nm[0]
    dy = all_positions[:,:,1] - cnt_center_nm[1]
    dz = all_positions[:,:,2]
    radii = np.sqrt(dx**2 + dy**2)
    inside_mask = (radii < R_tube_nm) & (dz > z_min_nm) & (dz < z_max_nm)

    n_waters = inside_mask.shape[1]
    msd3D_accum = np.zeros(n_frames)
    msdZ_accum  = np.zeros(n_frames)
    counts = np.zeros(n_frames, dtype=int)

    # Save inside-only trajectories
    inside_traj_start = []
    inside_traj_end   = []

    for trial in range(n_trials):
        t0 = rng.integers(0, n_frames - 10)
        inside_now = np.where(inside_mask[t0])[0]
        if len(inside_now) == 0:
            continue

        chosen = rng.choice(inside_now, size=min(n_mols_per_trial, len(inside_now)), replace=False)

        # --- Vectorized displacement calculation ---
        # Positions for chosen molecules starting at t0
        pos_ref = all_positions[t0, chosen]  # shape (n_chosen, 3)

        # For all future frames
        pos_future = all_positions[t0 + 1:, chosen]        # shape (n_future, n_chosen, 3)
        mask_future = inside_mask[t0 + 1:, chosen]         # shape (n_future, n_chosen)
        mask_start = inside_mask[t0, chosen][None, :]      # shape (1, n_chosen)
        inside_both = mask_future & mask_start             # molecules inside at both t0 and t0+dt

        # Compute all displacements from t0 at once
        delta = pos_future - pos_ref                       # shape (n_future, n_chosen, 3)
        delta_sq = np.sum(delta**2, axis=2)
        delta_z  = delta[:, :, 2]**2

        # Apply mask and average per Δt
        for dt in range(delta_sq.shape[0]):
            valid = inside_both[dt]
            if not np.any(valid):
                continue

            pos_t0 = pos_ref[valid]
            pos_dt = pos_future[dt, valid]

            # Store confined positions for visualization
            inside_traj_start.append(pos_t0)
            inside_traj_end.append(pos_dt)

            msd3D_accum[dt + 1] += np.mean(delta_sq[dt, valid])
            msdZ_accum[dt + 1]  += np.mean(delta_z[dt, valid])
            counts[dt + 1] += 1


    valid = counts > 0
    msd3D = np.zeros_like(msd3D_accum)
    msdZ  = np.zeros_like(msdZ_accum)
    msd3D[valid] = msd3D_accum[valid] / counts[valid]
    msdZ[valid]  = msdZ_accum[valid]  / counts[valid]

    # Combine trajectories for saving
    if inside_traj_start:
        inside_traj_start = np.vstack(inside_traj_start)
        inside_traj_end   = np.vstack(inside_traj_end)
    else:
        inside_traj_start = np.empty((0,3))
        inside_traj_end   = np.empty((0,3))

    # Save as NumPy arrays
    #np.save(os.path.join(save_path, "inside_traj_start.npy"), inside_traj_start)
    #np.save(os.path.join(save_path, "inside_traj_end.npy"), inside_traj_end)
    print(f"Saved {len(inside_traj_start)} confined trajectories to {save_path}")

    # Plot XY and XZ projections
    plt.figure(figsize=(5,5))
    plt.scatter(inside_traj_start[::10000,0], inside_traj_start[::10000,1], s=3, alpha=0.4, label='Start positions')
    plt.scatter(inside_traj_end[::10000,0], inside_traj_end[::10000,1], s=3, alpha=0.4, label='End positions')
    circle = plt.Circle(cnt_center_nm, R_tube_nm, color='r', fill=False, linestyle='--', label='CNT radius')
    plt.gca().add_patch(circle)
    plt.axis('equal')
    plt.xlabel('x (nm)')
    plt.ylabel('y (nm)')
    plt.legend()
    plt.title('Confined water trajectories (XY view)')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "Figures/inside_traj_XY.png"), dpi=600)
    plt.close()

    plt.figure(figsize=(6,5))
    plt.scatter(inside_traj_start[::10000,0], inside_traj_start[::10000,2], s=3, alpha=0.4, label='Start')
    plt.scatter(inside_traj_end[::10000,0], inside_traj_end[::10000,2], s=3, alpha=0.4, label='End')
    plt.axhline(z_min_nm, color='r', linestyle='--')
    plt.axhline(z_max_nm, color='r', linestyle='--', label='CNT bounds')
    plt.xlabel('x (nm)')
    plt.ylabel('z (nm)')
    plt.legend()
    plt.title('Confined water trajectories (XZ view)')
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, "Figures/inside_traj_XZ.png"), dpi=600)
    plt.close()

    print("Saved trajectory plots.")

    df = pd.DataFrame({
    "x": inside_traj_start[::10000,0],
    "y": inside_traj_start[::10000,1],
    "z": inside_traj_start[::10000,2],
    })

    # Save to CSV
    df.to_csv(os.path.join(path, "Figures", "water_locations_start.csv"), index=False)

    df = pd.DataFrame({
    "x": inside_traj_end[::10000,0],
    "y": inside_traj_end[::10000,1],
    "z": inside_traj_end[::10000,2],
    })

    # Save to CSV
    df.to_csv(os.path.join(path, "Figures", "water_locations_end.csv"), index=False)

    return time, msd3D, msdZ

def parse_nm(s):
    """
    Extract n and m from a CNT string like 'CNT_n06_m06_L50_200'
    """
    match = re.search(r"n(\d+)_m(\d+)", s)
    if not match:
        raise ValueError("Could not find n and m in the string.")
    n = int(match.group(1))
    m = int(match.group(2))
    return n, m


def cnt_diameter(n, m, a=0.246):  
    """
    Compute CNT diameter (in nm) from chiral indices (n, m).
    Formula: d = (a/pi) * sqrt(n^2 + n*m + m^2)
    """
    return (a / math.pi) * math.sqrt(n**2 + n*m + m**2)

def calculate_hbonds_confined_water(topology_path, trajectory_path):
    """
    Calculate the average number of hydrogen bonds per confined water molecule in a CNT system.
    
    Parameters:
    -----------
    topology_path : str
        Path to the topology file (e.g., .tpr file)
    trajectory_path : str
        Path to the trajectory file (e.g., .xtc file)
    
    Returns:
    --------
    float
        Average number of hydrogen bonds per confined water molecule
    """
    # Load the trajectory
    u = mda.Universe(topology_path, trajectory_path)
    
    # Identify carbon nanotube and water molecules
    cnt = u.select_atoms('resname CNT')
    water_oxygens = u.select_atoms('resname SOL and name OW')
    
    # Determine CNT geometry (center and radius)
    u.trajectory[0]
    cnt_center_xy = np.array([cnt.center_of_mass()[0], cnt.center_of_mass()[1]])
    
    # Calculate CNT radius
    radial_distances = np.sqrt((cnt.positions[:, 0] - cnt_center_xy[0])**2 + 
                               (cnt.positions[:, 1] - cnt_center_xy[1])**2)
    
    # Use conservative cutoff for internal cavity (slightly smaller than CNT wall radius)
    radius_cutoff = np.mean(radial_distances) - 1.0  # Conservative estimate
    
    # Function to identify water molecules inside CNT
    def get_water_inside_cnt(water_oxygens, cnt_center_xy, cnt_radius_cutoff):
        radial_dist = np.sqrt((water_oxygens.positions[:, 0] - cnt_center_xy[0])**2 + 
                              (water_oxygens.positions[:, 1] - cnt_center_xy[1])**2)
        inside_mask = radial_dist < cnt_radius_cutoff
        return inside_mask
    
    # Use last 100 frames for computational efficiency
    n_frames_to_use = 100
    start_frame = max(0, len(u.trajectory) - n_frames_to_use)
    
    # Track water molecules that remain inside CNT throughout the analysis period
    inside_molecules_per_frame = []
    
    for ts in u.trajectory[start_frame:]:
        inside_mask = get_water_inside_cnt(water_oxygens, cnt_center_xy, radius_cutoff)
        inside_indices = np.where(inside_mask)[0]
        inside_molecules_per_frame.append(set(inside_indices))
    
    # Find water molecules that are inside CNT in ALL frames (never escape)
    confined_molecules = inside_molecules_per_frame[0]
    for frame_set in inside_molecules_per_frame[1:]:
        confined_molecules = confined_molecules.intersection(frame_set)
    
    confined_molecules = sorted(list(confined_molecules))
    
    # Get residue IDs of confined waters
    u.trajectory[start_frame]
    confined_oxygen_indices = np.array(confined_molecules)
    confined_water_resids = water_oxygens[confined_oxygen_indices].resids
    
    # Create selection string for confined waters
    confined_water_selection = f"resname SOL and resid {' '.join(map(str, confined_water_resids))}"
    
    # Perform hydrogen bond analysis on confined water molecules
    hbonds = HydrogenBondAnalysis(
        universe=u,
        donors_sel=confined_water_selection + " and name OW",
        hydrogens_sel=confined_water_selection + " and name HW*",
        acceptors_sel=confined_water_selection + " and name OW",
        d_h_cutoff=1.2,
        d_a_cutoff=3.5,
        d_h_a_angle_cutoff=150
    )

    
    # Run analysis on the selected frames
    hbonds.run(start=start_frame, stop=len(u.trajectory), step=1, verbose=False)
    
    # Extract hydrogen bond statistics
    hbond_results = hbonds.results.hbonds
    
    # Count hydrogen bonds per frame
    hbonds_per_frame = []
    for frame_num in range(start_frame, len(u.trajectory)):
        frame_hbonds = hbond_results[hbond_results[:, 0] == frame_num]
        hbonds_per_frame.append(len(frame_hbonds))
    
    hbonds_per_frame = np.array(hbonds_per_frame)
    average_hbonds = np.mean(hbonds_per_frame)
    
    # Calculate average number of hydrogen bonds per water molecule
    num_confined_waters = len(confined_water_resids)
    avg_hbonds_per_molecule = average_hbonds / num_confined_waters
    
    return avg_hbonds_per_molecule




## Run Analysis on MD Simulation Files 

In [ ]:
path1 = "../Data/Reproducibility_5ns_2/"
filenames = os.listdir(path1)
filenames

['CNT_n06_m06_L50_350',
 'CNT_n07_m07_L50_290',
 'CNT_n09_m09_L50_262',
 'CNT_n11_m10_L50_248']

In [8]:
filenames = os.listdir(path1)
for i in range(len(filenames)):
    path = path1 + filenames[i] + '/'
    TOPO = path + "nvt.tpr"
    TRAJ = path + "nvt_whole.xtc"
    os.makedirs(path + "Figures", exist_ok=True)
    n, m = parse_nm(filenames[i])
    diameter = cnt_diameter(n, m)
    u = mda.Universe(TOPO, TRAJ)
    x = u.trajectory[0].dimensions[0]
    y = u.trajectory[0].dimensions[1]
    cnt_center = np.array([x/2, y/2])   # [x0, y0] Å
    #cnt_center = np.array([15, 15])
    R_tube = (diameter*10)/2                          # CNT radius (Å)
    ############ For the 14 nm long tube ####################
    z_min, z_max = 30, 480.0              # CNT ends (Å) the z distance of the ends of the CNT
    z_min_core, z_max_core = 3.0, 48.0    # nm (avoid ends) the distance cutoff to be considered inside the tube change back to 14 for the 14nm tube
    fit_start = 100
    fit_end = 1200

    u, waters = load_trajectory(TOPO, TRAJ)

    time, msd3D, msdZ = compute_confined_msd_random_with_traj(u, waters, cnt_center, R_tube, z_min, z_max,
                                            n_trials=3000, n_mols_per_trial=200, seed=None,
                                            save_path=path)

    # Fit and extract diffusion coefficients
    D_3D, D_3D_cm2_s, D_z, D_z_cm2_s, slope_3D, intercept_3D, slope_z, intercept_z, mask = \
        fit_diffusion_coefficients(time, msd3D, msdZ, fit_start, fit_end)

    # Save results
    save_diffusion_results(path, time, msd3D, msdZ, D_3D_cm2_s, D_z_cm2_s)

    # Plot MSD fits
    plot_msd(time, msd3D, msdZ, slope_3D, intercept_3D, slope_z, intercept_z, mask, path)

    # Plot trajectories
    #plot_radial_axial_positions(time, radii, z_positions, R_tube/10, z_min_core, z_max_core, path)

    # 3D Visualization
    plot_3D_snapshot(u, waters, cnt_center/10, R_tube/10, z_min/10, z_max/10, z_min_core, z_max_core, path)

    # Radial density
    compute_radial_density(u, cnt_center/10, R_tube/10, z_min/10, z_max/10, path)

    # Bulk water density
    compute_bulk_density(u, cnt_center/10, R_tube/10, path)

    # Calculate hydrogen bonds 
    bonds_per_molecule = calculate_hbonds_confined_water(TOPO, TRAJ)
    df = pd.DataFrame({bonds_per_molecule}, columns=['Average Number of H-Bonds'])
    df.to_csv(path + '/Figures/H_Bonds.csv', index=False)

    print('File Number:', str(i))

Reading trajectory: 100%|██████████| 834/834 [00:00<00:00, 930.15it/s]


Saved 200882073 confined trajectories to ../Data/Reproducibility_5ns_2/CNT_n06_m06_L50_350/
Saved trajectory plots.

--- Confined-water MSD (permanent escape removal) ---
3D D = 8.493e-04 nm²/ps  (8.493e-06 cm²/s)
z-axis D = 2.548e-03 nm²/ps  (2.548e-05 cm²/s)

Generating 3D visualization for final frame...
Average bulk water density (outside CNT): 0.885 g/cm³
File Number: 0


Reading trajectory: 100%|██████████| 834/834 [00:01<00:00, 675.73it/s]


Saved 244011814 confined trajectories to ../Data/Reproducibility_5ns_2/CNT_n07_m07_L50_290/
Saved trajectory plots.

--- Confined-water MSD (permanent escape removal) ---
3D D = 6.210e-04 nm²/ps  (6.210e-06 cm²/s)
z-axis D = 1.863e-03 nm²/ps  (1.863e-05 cm²/s)

Generating 3D visualization for final frame...
Average bulk water density (outside CNT): 0.886 g/cm³
File Number: 1


Reading trajectory: 100%|██████████| 834/834 [00:01<00:00, 815.89it/s]


Saved 252449826 confined trajectories to ../Data/Reproducibility_5ns_2/CNT_n09_m09_L50_262/
Saved trajectory plots.

--- Confined-water MSD (permanent escape removal) ---
3D D = 4.696e-05 nm²/ps  (4.696e-07 cm²/s)
z-axis D = 1.374e-04 nm²/ps  (1.374e-06 cm²/s)

Generating 3D visualization for final frame...
Average bulk water density (outside CNT): 0.833 g/cm³
File Number: 2


Reading trajectory: 100%|██████████| 834/834 [00:01<00:00, 777.57it/s]


Saved 241280887 confined trajectories to ../Data/Reproducibility_5ns_2/CNT_n11_m10_L50_248/
Saved trajectory plots.

--- Confined-water MSD (permanent escape removal) ---
3D D = 6.437e-04 nm²/ps  (6.437e-06 cm²/s)
z-axis D = 1.931e-03 nm²/ps  (1.931e-05 cm²/s)

Generating 3D visualization for final frame...
Average bulk water density (outside CNT): 0.787 g/cm³
File Number: 3
